In [2]:
%pip list

Package                      Version
---------------------------- --------------
absl-py                      2.1.0
anyio                        4.6.2.post1
argon2-cffi                  23.1.0
argon2-cffi-bindings         21.2.0
arrow                        1.3.0
asttokens                    2.4.1
astunparse                   1.6.3
async-lru                    2.0.4
attrs                        24.2.0
babel                        2.16.0
beautifulsoup4               4.12.3
bleach                       6.2.0
cachetools                   5.5.0
certifi                      2024.8.30
cffi                         1.17.1
charset-normalizer           3.4.0
colorama                     0.4.6
comm                         0.2.2
debugpy                      1.8.9
decorator                    5.1.1
defusedxml                   0.7.1
executing                    2.1.0
fastjsonschema               2.20.0
flatbuffers                  24.3.25
fqdn                         1.5.1
gast                     

In [ ]:
import os

import numpy as np
import tensorflow as tf
print(tf.__version__)

NUM_FEATURES = 266
NUM_CLASSES = 2


class CustomModel(tf.Module):
  def __init__(self, learning_rate=0.001):
    self.num_features = NUM_FEATURES
    self.num_classes = NUM_CLASSES
    self.model = tf.keras.Sequential([
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', name='dense_1'),
    ])
    self.model.build(input_shape=[None, NUM_FEATURES])
    self.model.compile(
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=tf.keras.losses.CategoricalCrossentropy())
    
  @tf.function(input_signature=[
      tf.TensorSpec([None, NUM_FEATURES], tf.float32),
      tf.TensorSpec([None, NUM_CLASSES], tf.float32),
  ])
  def train(self, x, y):
    with tf.GradientTape() as tape:
      prediction = self.model(x)
      loss = self.model.loss(y, prediction)
      loss = tf.cond(tf.math.is_nan(loss) | (loss < 0), 
                               lambda: tf.constant(1e-6, dtype=tf.float32), 
                               lambda: loss)  
    gradients = tape.gradient(loss, self.model.trainable_variables)
    self.model.optimizer.apply_gradients(
        zip(gradients, self.model.trainable_variables))
    result = {"loss": loss}
    return result

  @tf.function(input_signature=[
      tf.TensorSpec([None, NUM_FEATURES], tf.float32),
  ])
  def infer(self, x):
    logits = self.model(x)
    probabilities = tf.nn.softmax(logits, axis=-1)
    return {
        "output": probabilities,
        "logits": logits
    }

  @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
  def save(self, checkpoint_path):
    tensor_names = [item.name for item in self.model.layers[0].trainable_variables]
    tensors_to_save = [item.read_value() for item in self.model.layers[0].trainable_variables]
    tf.raw_ops.Save(
        filename=checkpoint_path,
        tensor_names=tensor_names,
        data=tensors_to_save,
        name='save')
    return {'checkpoint_path': checkpoint_path}

  @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
  def restore(self, checkpoint_path):
    restored_tensors = {}
    for var in self.model.weights:
      restored = tf.raw_ops.Restore(
          file_pattern=checkpoint_path, tensor_name=var.name, dt=var.dtype,
          name='restore')
      var.assign(restored)
      restored_tensors[var.name] = restored
    for var in self.model.layers[0].trainable_variables:
      restored = tf.raw_ops.Restore(
          file_pattern=checkpoint_path, tensor_name=var.name, dt=var.dtype,
          name='restore')
    return restored_tensors




def convert_and_save(saved_model_dir='saved_model'):

  model = CustomModel()

  tf.saved_model.save(
      model,
      saved_model_dir,
      signatures={
          'train': model.train.get_concrete_function(),
          'infer': model.infer.get_concrete_function(),
          'save': model.save.get_concrete_function(),
          'restore': model.restore.get_concrete_function(),
      })

  # Convert the model
  converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
  converter.target_spec.supported_ops = [
      tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
      tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
  ]
  converter.experimental_enable_resource_variables = True
  tflite_model = converter.convert()

  model_name = 'CustomModel' + '_' + str(NUM_FEATURES) + 'x' + str(NUM_CLASSES)

  model_file_path = os.path.join(model_name + '.tflite')
  with open(model_file_path, 'wb') as model_file:
    model_file.write(tflite_model)


if __name__ == '__main__':
  convert_and_save()

2.12.0


INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets
